## funções

In [17]:
def add_ner_to_dic(term_list, ner_rules_dic, ner_entity = None, ner_level = 0):
    
    print(f'Adicionando entidades no ner_rules_dic[{ner_entity}]...')
    
    try:
        initial_len_terms = len(ner_rules_dic[ner_entity]['terms'])
    except KeyError:
        initial_len_terms = 0

    #checando se a entidade já existe no ner_rules
    try:
        ner_rules_dic[ner_entity]
    
    except KeyError:
        ner_rules_dic[ner_entity] = {}
        ner_rules_dic[ner_entity][ 'terms' ] = []
        ner_rules_dic[ner_entity][ 'level' ] = ner_level
        
    for term in term_list:
        if term not in ner_rules_dic[ner_entity]['terms']:
            ner_rules_dic[ner_entity][ 'terms' ].append(term)
        
    ner_rules_dic[ner_entity][ 'terms' ].sort()

    #sorting dict
    myKeys = list(ner_rules_dic.keys())
    myKeys.sort()
    ner_rules_dic = {i: ner_rules_dic[i] for i in myKeys}

    print('Foram adicionados ', len(ner_rules_dic[ner_entity]['terms']) - initial_len_terms, ' termos (total: ', len(ner_rules_dic[ner_entity]['terms']), ')' )
    
    return ner_rules_dic

In [18]:
def add_to_ngrams_to_remove(term_list, ngrams_to_replace_dic, cat = None):
    
    print(f'Adicionando entidades no ngrams_to_replace[{cat}]...')
    
    try:
        initial_len_terms = len(ngrams_to_replace_dic[cat])
    except KeyError:
        initial_len_terms = 0

    #checando se a entidade já existe no ngrams_to_replace_dic
    try:
        ngrams_to_replace_dic[cat]
    
    except KeyError:
        ngrams_to_replace_dic[cat] = []
        
    for term in term_list:
        if term not in ngrams_to_replace_dic[cat]:
            ngrams_to_replace_dic[cat].append(term)
        
    ngrams_to_replace_dic[cat].sort()

    #sorting dict
    myKeys = list(ngrams_to_replace_dic.keys())
    myKeys.sort()
    ngrams_to_replace_dic = {i: ngrams_to_replace_dic[i] for i in myKeys}

    print('Foram adicionados ', len(ngrams_to_replace_dic[cat]) - initial_len_terms, ' termos (total: ', len(ngrams_to_replace_dic[cat]), ')' )
    
    return ngrams_to_replace_dic

In [19]:
def combine_entities(first_entity, second_entity, ner_rules_dic = None, new_entity_name = None, lower_ngrams = False):
    
    print(f'Combinando entidades no ner_rules_dic[{new_entity_name}]: ', first_entity, ' + ', second_entity)

    try:
        initial_len_terms = len(ner_rules_dic[new_entity_name]['terms'])
    except KeyError:
        initial_len_terms = 0

    #checando se a entidade já existe no ner_rules
    try:
        ner_rules_dic[new_entity_name]
    
    except KeyError:
        ner_rules_dic[new_entity_name] = {}
        ner_rules_dic[new_entity_name][ 'terms' ] = []

        new_level = ner_rules_dic[first_entity]['level'] + ner_rules_dic[second_entity]['level'] + 1
        ner_rules_dic[new_entity_name]['level'] = new_level
    
    for i in ner_rules_dic[first_entity]['terms']:
        for j in ner_rules_dic[second_entity]['terms']:
            
            if lower_ngrams is True:
                new_term = i.lower() + ' ' + j.lower()
            else:
                new_term = i + ' ' + j
            
            if new_term not in ner_rules_dic[new_entity_name]['terms']:
                ner_rules_dic[new_entity_name]['terms'].append( new_term )

    ner_rules_dic[new_entity_name]['terms'].sort()

    #sorting dict
    myKeys = list(ner_rules_dic.keys())
    myKeys.sort()
    ner_rules_dic = {i: ner_rules_dic[i] for i in myKeys}

    print('Key modificada: ', new_entity_name)
    print('Foram adicionados ', len(ner_rules_dic[new_entity_name]['terms']) - initial_len_terms, ' termos (total: ', len(ner_rules_dic[new_entity_name]['terms']), ')' )

    return ner_rules_dic

In [20]:
def concatenate_entities(first_entity, second_entity, ner_rules_dic = None, new_entity_name = None, lower_ngrams = False):

    if new_entity_name is not None:
        concat_entity_name = new_entity_name
    else: 
        concat_entity_name = first_entity

    print(f'Concatenando entidades no ner_rules_dic[{concat_entity_name}]: ', first_entity, ' + ', second_entity)

    try:
        initial_len_terms = len(ner_rules_dic[concat_entity_name]['terms'])
    except KeyError:
        initial_len_terms = 0

    #checando se a entidade já existe no ner_rules
    try:
        ner_rules_dic[concat_entity_name]
    
    except KeyError:
        ner_rules_dic[concat_entity_name] = {}
        ner_rules_dic[concat_entity_name][ 'terms' ] = []

        new_level = ner_rules_dic[first_entity]['level'] + ner_rules_dic[second_entity]['level']
        ner_rules_dic[concat_entity_name]['level'] = new_level
    
    for i in ner_rules_dic[first_entity]['terms']:
        if i not in ner_rules_dic[concat_entity_name]['terms']:
            if lower_ngrams is True:
                i = i.lower()
            ner_rules_dic[concat_entity_name]['terms'].append( i )
    
    for j in ner_rules_dic[second_entity]['terms']:
        if j not in ner_rules_dic[concat_entity_name]['terms']:
            if lower_ngrams is True:
                j = j.lower()
            ner_rules_dic[concat_entity_name]['terms'].append( j )

    ner_rules_dic[concat_entity_name]['terms'].sort()

    #sorting dict
    myKeys = list(ner_rules_dic.keys())
    myKeys.sort()
    ner_rules_dic = {i: ner_rules_dic[i] for i in myKeys}

    print('Key modificada: ', concat_entity_name)
    print('Foram adicionados ', len(ner_rules_dic[concat_entity_name]['terms']) - initial_len_terms, ' termos (total: ', len(ner_rules_dic[concat_entity_name]['terms']), ')' )

    return ner_rules_dic

In [21]:
import json
import time
import os
import regex as re
import pandas as pd

_______________________________________________________________________________________________________________

## abrindo e dando sort no ner_rules.json

In [24]:
#abrindo o dicionário com os ner rules
with open('ner_rules.json', 'r') as file:
    ner_dic = json.load(file)
    file.close()

#sorting dict
classes = list(ner_dic.keys())
classes.sort()
new_ner_dic = {}
terms = {}
for i in classes:
    new_ner_dic[i] = {}
    new_ner_dic[i]['level'] = ner_dic[i]['level']
    
    terms = list(ner_dic[i]['terms'])
    terms.sort()
    new_ner_dic[i]['terms'] = terms

ner_dic = new_ner_dic

print('> Classes:')
for key in ner_dic.keys():
    print(key, '; termos: ', len(ner_dic[key]['terms']))

> Classes:
2D materials name 1gram ; termos:  6
2D materials symbol ; termos:  30
carbon material name 1gram ; termos:  7
chemical element name ; termos:  118
chemical element symbol ; termos:  118
metallic_nano element name ; termos:  21
metallic_nano element symbol ; termos:  21
nanomaterial morphology ; termos:  30
oxides symbol ; termos:  37
photosensitizers ; termos:  149
quantum dots symbol ; termos:  21
species daphnia ; termos:  12
species genus all abbr ; termos:  613
species genus all full ; termos:  683


_______________________________________________________________________________________________________________

## adicionando novas entradas de TXT ao ner_rules.json

In [26]:
ner_level = 0
ner_entity_name = 'toxicity'

#abrindo o dicionário com os ner rules
with open('ner_rules.json', 'r') as ner_json_file:
    ner_dic = json.load(ner_json_file)
    ner_json_file.close()

l = []
with open(r'C:\Users\Dell\Dropbox\SolBIN\Modelos Python\aRIX engine v2\Inputs\\add_these.txt') as f:
    for line in f.readlines():
        term = line[ : -1] if line[ -1 : ] == '\n' else line[ : ]
        l.append( term )
    f.close()

#adicionando uma lista de termos ao dicinário com os ner_rules
ner_dic = add_ner_to_dic(l, ner_dic, ner_entity = ner_entity_name, ner_level = ner_level)

ner_dic[ner_entity_name]

Adicionando entidades no ner_rules_dic[toxicity]...
Foram adicionados  19  termos (total:  19 )


{'terms': ['benign effect',
  'detrimental effect',
  'elevated levels of toxicity',
  'high toxicity',
  'highly toxic',
  'little to no adverse effects',
  'little toxicity',
  'low levels of toxicity',
  'minimal signs of toxicity',
  'minimal toxicity',
  'negligible toxicity',
  'no significant toxicity',
  'no toxicity',
  'non-toxic',
  'notably toxic',
  'pronounced toxicity',
  'severe adverse effects',
  'significant toxicity',
  'signs of toxicity'],
 'level': 0}

_______________________________________________________________________________________________________________

## combinando entities do ner_rules.json

In [98]:
ner_entity_name = 'temp nanomaterials'

#abrindo o dicionário com os ner rules
with open('ner_rules.json', 'r') as ner_json_file:
    ner_dic = json.load(ner_json_file)
    ner_json_file.close()

#combinando as entidades para formar uma nova
#adicionando uma lista de termos ao dicinário com os ner_rules
ner_dic = combine_entities('carbon nanomaterial name', 'nanomaterial morphology', ner_rules_dic = ner_dic, new_entity_name = ner_entity_name, lower_ngrams = False)

ner_dic[ner_entity_name]

Combinando entidades no ner_rules_dic[temp nanomaterials]:  carbon nanomaterial name  +  nanomaterial morphology
Key modificada:  temp nanomaterials
Foram adicionados  45  termos (total:  1020 )


{'terms': ['BN dots',
  'BN nanocapsule',
  'BN nanocube',
  'BN nanodisk',
  'BN nanofiber',
  'BN nanohelix',
  'BN nanoparticle',
  'BN nanoribbon',
  'BN nanorod',
  'BN nanosheet',
  'BN nanosphere',
  'BN nanostar',
  'BN nanostructure',
  'BN nanotube',
  'BN nanowire',
  'CrSe2 dots',
  'CrSe2 nanocapsule',
  'CrSe2 nanocube',
  'CrSe2 nanodisk',
  'CrSe2 nanofiber',
  'CrSe2 nanohelix',
  'CrSe2 nanoparticle',
  'CrSe2 nanoribbon',
  'CrSe2 nanorod',
  'CrSe2 nanosheet',
  'CrSe2 nanosphere',
  'CrSe2 nanostar',
  'CrSe2 nanostructure',
  'CrSe2 nanotube',
  'CrSe2 nanowire',
  'HfSe2 dots',
  'HfSe2 nanocapsule',
  'HfSe2 nanocube',
  'HfSe2 nanodisk',
  'HfSe2 nanofiber',
  'HfSe2 nanohelix',
  'HfSe2 nanoparticle',
  'HfSe2 nanoribbon',
  'HfSe2 nanorod',
  'HfSe2 nanosheet',
  'HfSe2 nanosphere',
  'HfSe2 nanostar',
  'HfSe2 nanostructure',
  'HfSe2 nanotube',
  'HfSe2 nanowire',
  'Mo2C dots',
  'Mo2C nanocapsule',
  'Mo2C nanocube',
  'Mo2C nanodisk',
  'Mo2C nanofiber',

_______________________________________________________________________________________________________________

## concatenando entities do ner_rules.json

In [113]:
ner_entity_name = 'temp quantum dots'

#abrindo o dicionário com os ner rules
with open('ner_rules.json', 'r') as ner_json_file:
    ner_dic = json.load(ner_json_file)
    ner_json_file.close()

#combinando as entidades para formar uma nova
#adicionando uma lista de termos ao dicinário com os ner_rules
ner_dic = concatenate_entities('quantum dots name', 'quantum dots symbol', ner_rules_dic = ner_dic, new_entity_name = ner_entity_name, lower_ngrams = True)

Concatenando entidades no ner_rules_dic[temp quantum dots]:  quantum dots name  +  quantum dots symbol
Key modificada:  temp quantum dots
Foram adicionados  42  termos (total:  42 )


_______________________________________________________________________________________________________________

## modificar termos do ner_rules.json com padrões regex

In [23]:
ner_level = 1
new_ner_entity_name = 'species regex regex all'
ner_entity_do_add_regex = 'species regex all'

#abrindo o dicionário com os ner rules
with open('ner_rules.json', 'r') as ner_json_file:
    ner_dic = json.load(ner_json_file)
    ner_json_file.close()
    
l = []
for term in ner_dic[ner_entity_do_add_regex]['terms']:
    new_term = r'[A-Z][a-z]+\s' + term
    l.append(new_term)

#adicionando uma lista de termos ao dicinário com os ner_rules
ner_dic = add_ner_to_dic(l, ner_dic, ner_entity = new_ner_entity_name, ner_level = ner_level)

ner_dic[new_ner_entity_name]

Adicionando entidades no ner_rules_dic[species regex regex all]...
Foram adicionados  822  termos (total:  822 )


{'terms': ['[A-Z][a-z]+\\sa2',
  '[A-Z][a-z]+\\saacr',
  '[A-Z][a-z]+\\sabdomen',
  '[A-Z][a-z]+\\sabscessus',
  '[A-Z][a-z]+\\sabstract',
  '[A-Z][a-z]+\\sacademic',
  '[A-Z][a-z]+\\sacidophilus',
  '[A-Z][a-z]+\\sacinetobacter',
  '[A-Z][a-z]+\\sacnes',
  '[A-Z][a-z]+\\sactinobacillus',
  '[A-Z][a-z]+\\sactinomyces',
  '[A-Z][a-z]+\\sactinomycetemcomitans',
  '[A-Z][a-z]+\\sacuminata',
  '[A-Z][a-z]+\\sacuminatum',
  '[A-Z][a-z]+\\sadoptively',
  '[A-Z][a-z]+\\sae',
  '[A-Z][a-z]+\\saerogenes',
  '[A-Z][a-z]+\\saeruginosa',
  '[A-Z][a-z]+\\sagar',
  '[A-Z][a-z]+\\sagglutinin',
  '[A-Z][a-z]+\\saggregatibacter',
  '[A-Z][a-z]+\\saia',
  '[A-Z][a-z]+\\saip',
  '[A-Z][a-z]+\\saipaam',
  '[A-Z][a-z]+\\sakt',
  '[A-Z][a-z]+\\salanine',
  '[A-Z][a-z]+\\salbicans',
  '[A-Z][a-z]+\\salkaloid',
  '[A-Z][a-z]+\\salkylating',
  '[A-Z][a-z]+\\sallo',
  '[A-Z][a-z]+\\sallograft',
  '[A-Z][a-z]+\\saloh',
  '[A-Z][a-z]+\\salsh',
  '[A-Z][a-z]+\\salternata',
  '[A-Z][a-z]+\\samazonensis',
  '[A-Z][a

_______________________________________________________________________________________________________________

## removendo alguma key do ner_rules.json

In [7]:
ner_entity_name_to_remove = 'teste'

#abrindo o dicionário com os ner rules
with open('ner_rules.json', 'r') as ner_json_file:
    ner_dic = json.load(ner_json_file)
    ner_json_file.close()

del ner_dic[ner_entity_name_to_remove]
print('> Excluido: ', ner_entity_name_to_remove)

print('> Classes:')
for key in ner_dic.keys():
    print(key, '; termos: ', len(ner_dic[key]['terms']))

> Excluido:  teste
> Classes:
2D materials name 1gram ; termos:  6
2D materials symbol ; termos:  30
carbon material name 1gram ; termos:  6
chemical element name ; termos:  118
chemical element symbol ; termos:  118
metallic_nano element name ; termos:  21
metallic_nano element symbol ; termos:  21
nanomaterial morphology ; termos:  30
oxides symbol ; termos:  36
photosensitizers ; termos:  149
quantum dots symbol ; termos:  21
species daphnia ; termos:  12
species genus all abbr ; termos:  613
species genus all full ; termos:  683


_______________________________________________________________________________________________________________

## salvando o ner_rules.json

In [27]:
print('> Classes:')
for key in ner_dic.keys():
    print(key, '; termos: ', len(ner_dic[key]['terms']))

print('Deseja salvar o arquivo ner_rules.json? (s/n)')
time.sleep(3)
answer = input()
if answer.lower() == 's': 
    with open('ner_rules.json', 'w') as ner_json_file:
        json.dump(ner_dic, ner_json_file, indent=2)
        ner_json_file.close()

> Classes:
2D materials name 1gram ; termos:  6
2D materials symbol ; termos:  30
carbon material name 1gram ; termos:  7
chemical element name ; termos:  118
chemical element symbol ; termos:  118
metallic_nano element name ; termos:  21
metallic_nano element symbol ; termos:  21
nanomaterial morphology ; termos:  30
oxides symbol ; termos:  37
photosensitizers ; termos:  149
quantum dots symbol ; termos:  21
species daphnia ; termos:  12
species genus all abbr ; termos:  613
species genus all full ; termos:  683
toxicity ; termos:  19
Deseja salvar o arquivo ner_rules.json? (s/n)


_______________________________________________________________________________________________________________

## manipulando o ner_rules.json para pegar as espécies

In [11]:
l = []
for term in ner_dic['species genus all']['terms']:
    f = term.split()[1]
    if f not in l:
        l.append(f)
l.sort()
l

pd.DataFrame(l).to_csv('add_these.csv')

_______________________________________________________________________________________________________________

## manipulando o ner_rules.json para pegar abbreviações para o ngrams_to_replace.json

In [9]:
l = []
dic_to_replace = {}
for term in ner_dic['species genus all full']['terms']:
    f = term[0] + '. '
    f += term.split()[1]
    if f not in l:
        l.append(f)
        dic_to_replace[f] = term
    else:
        del l[ l.index(f) ]
        del dic_to_replace[f]

print( len(l), len(dic_to_replace) )
dic_to_replace

pd.DataFrame(l).to_csv('add_to_ner_dic.csv')
with open('add_to_replace.json', 'w') as file:
    json.dump(dic_to_replace, file, indent=2)
    file.close()

614 614


_______________________________________________________________________________________________________________

## abrindo e dando sort no ngrams_to_replace.json

In [28]:
#abrindo o dicionário com os ner rules
with open('ngrams_to_replace.json', 'r') as file:
    ngrams_to_replace_dic = json.load(file)
    file.close()

#sorting dict
classes = list(ngrams_to_replace_dic.keys())
classes.sort()
terms_to_replace = {}
for i in classes:
    terms = list(ngrams_to_replace_dic[i].keys())
    terms.sort()

    terms_to_replace[i] = {j: ngrams_to_replace_dic[i][j] for j in terms}
    
ngrams_to_replace_dic = {i: terms_to_replace[i] for i in classes}

print('> Classes:')
for key in ngrams_to_replace_dic.keys():
    print(key, '; termos: ', len(ngrams_to_replace_dic[key]))

> Classes:
applications ; termos:  10
elements name symbol ; termos:  236
extract species ; termos:  615
inorganic compounds name symbol ; termos:  94
nanomaterial morphology ; termos:  42
photosentitizers ; termos:  1
processes materials ; termos:  122
raw materials ; termos:  16
toxicity ; termos:  19
zebrafish ; termos:  3


_______________________________________________________________________________________________________________

## Incorporando novas entradas ao ngrams_to_replace.json por arquivo

In [13]:
#abrindo o dicionário com os ner rules
with open('ngrams_to_replace.json', 'r') as file:
    ngrams_to_replace_dic = json.load(file)
    file.close()

add_to_key = 'elements name symbol'
new_dic_filename = 'add_these.json'
with open(new_dic_filename) as json_file:
    new_dic = json.load(json_file)
    json_file.close()

#caso já exista a key
try:
    ngrams_to_replace_dic[add_to_key]
    k_initial = len(ngrams_to_replace_dic[add_to_key].keys())
    print(f'> Já existem {k_initial} entradas na key {add_to_key}.')
    for key in new_dic.keys():
        if key not in ngrams_to_replace_dic[add_to_key].keys():
            ngrams_to_replace_dic[add_to_key][key] = new_dic[key]
    print(f'> Foram adicionadas {len(ngrams_to_replace_dic[add_to_key].keys()) - k_initial} entradas na key {add_to_key}.')

except KeyError:
    ngrams_to_replace_dic[add_to_key] = new_dic

print('> Classes:')
for key in ngrams_to_replace_dic.keys():
    print(key, '; termos: ', len(ngrams_to_replace_dic[key]))

> Classes:
applications ; termos:  10
extract species ; termos:  615
inorganic compounds name symbol ; termos:  94
nanomaterial morphology ; termos:  42
photosentitizers ; termos:  1
processes materials ; termos:  122
raw materials ; termos:  16
zebrafish ; termos:  3
elements name symbol ; termos:  236


_______________________________________________________________________________________________________________

## manipulando novas entradas ao ngrams_to_replace.json

In [110]:
ner_dic_key_to_get = 'extract nanomaterials name'
key_to_add_ngrams_to_replace = 'extract nanomaterials name'
ngrams_to_replace_entry = 'composition name symbol'


#abrindo o dicionário com os ngrams to replace
with open('ngrams_to_replace.json', 'r') as file:
    ngrams_to_replace_dic = json.load(file)
    file.close()

#abrindo o dicionário com os ner rules
with open('ner_rules.json', 'r') as ner_json_file:
    ner_dic = json.load(ner_json_file)
    ner_json_file.close()


ngrams_to_replace_dic[key_to_add_ngrams_to_replace] = {}

for term in ner_dic[ner_dic_key_to_get]['terms']:
    found_to_replace = False
    if len(term.split()) == 2:
        try:
            m_term = ngrams_to_replace_dic[ngrams_to_replace_entry][term.split()[0]]
            last_term = term.split()[1]
            found_to_replace = True
        except KeyError:
            pass

    if len(term.split()) == 3:
        try:
            m_term = ngrams_to_replace_dic[ngrams_to_replace_entry][term.split()[0] + ' ' + term.split()[1]]
            last_term = term.split()[2]
            found_to_replace = True
        except KeyError:
            pass
    
    if len(term.split()) == 4:
        try:
            m_term = ngrams_to_replace_dic[ngrams_to_replace_entry][term.split()[0] + ' ' + term.split()[1] + ' ' + term.split()[2]]
            last_term = term.split()[3]
            found_to_replace = True
        except KeyError:
            pass

    if len(term.split()) == 5:
        try:
            m_term = ngrams_to_replace_dic[ngrams_to_replace_entry][term.split()[0] + ' ' + term.split()[1] + ' ' + term.split()[2] + ' ' + term.split()[3]]
            last_term = term.split()[4]
            found_to_replace = True
        except KeyError:
            pass
        
    
    if found_to_replace is True:   
        modity_term = m_term + ' ' + last_term
        ngrams_to_replace_dic[key_to_add_ngrams_to_replace][term] = modity_term


print('> Classes:')
for key in ngrams_to_replace_dic.keys():
    print(key, '; termos: ', len(ngrams_to_replace_dic[key]))

> Classes:
applications ; termos:  10
composition name symbol ; termos:  89
composition symbol name ; termos:  90
photosentitizers ; termos:  1
processes materials ; termos:  122
raw materials ; termos:  16
species to extract ; termos:  615
zebrafish ; termos:  3
extract nanomaterials name ; termos:  1245


_______________________________________________________________________________________________________________

## removendo alguma key do ngrams_to_replace.json

In [10]:
ner_entity_name_to_remove = 'elements name symbol'

#abrindo o dicionário com os ner rules
with open('ngrams_to_replace.json', 'r') as ner_json_file:
    ngrams_to_replace_dic = json.load(ner_json_file)
    ner_json_file.close()

del ngrams_to_replace_dic[ner_entity_name_to_remove]
print('> Excluido: ', ner_entity_name_to_remove)

print('> Classes:')
for key in ngrams_to_replace_dic.keys():
    print(key, '; termos: ', len(ngrams_to_replace_dic[key]))

> Excluido:  elements name symbol
> Classes:
applications ; termos:  10
extract species ; termos:  615
inorganic compounds name symbol ; termos:  94
nanomaterial morphology ; termos:  42
photosentitizers ; termos:  1
processes materials ; termos:  122
raw materials ; termos:  16
zebrafish ; termos:  3


_______________________________________________________________________________________________________________

## salvando o ngrams_to_replace.json

In [29]:
print('> Classes:')
for key in ngrams_to_replace_dic.keys():
    print(key, '; termos: ', len(ngrams_to_replace_dic[key]))
    
print('Deseja salvar o arquivo ngrams_to_replace.json? (s/n)')
time.sleep(3)
answer = input()
if answer.lower() == 's': 
    with open('ngrams_to_replace.json', 'w') as json_file:
        json.dump(ngrams_to_replace_dic, json_file, indent=2)
        json_file.close()

> Classes:
applications ; termos:  10
elements name symbol ; termos:  236
extract species ; termos:  615
inorganic compounds name symbol ; termos:  94
nanomaterial morphology ; termos:  42
photosentitizers ; termos:  1
processes materials ; termos:  122
raw materials ; termos:  16
toxicity ; termos:  19
zebrafish ; termos:  3
Deseja salvar o arquivo ngrams_to_replace.json? (s/n)


_______________________________________________________________________________________________________________

## abrindo e dando sort no ngrams_to_remove.json

In [15]:
#abrindo o dicionário com os ner rules
with open('ngrams_to_remove.json', 'r') as file:
    ngrams_to_remove = json.load(file)
    file.close()

#sorting dict
classes = list(ngrams_to_remove.keys())
classes.sort()
new_ngrams_to_replace_dic = {}
for i in classes:
    terms = list(ngrams_to_remove[i])
    terms.sort()
    new_ngrams_to_replace_dic[i] = terms

ngrams_to_remove = new_ngrams_to_replace_dic

print('> Classes:')
for key in ngrams_to_remove.keys():
    print(key, '; termos: ', len(ngrams_to_remove[key]))

> Classes:
applications ; termos:  22
extract species ; termos:  4
processes ; termos:  33
publication Year ; termos:  1
raw_materials ; termos:  21


_______________________________________________________________________________________________________________

## adicionando novas entradas de TXT ao ngrams_to_remove.json

In [29]:
cat_name = 'extract species'

l = []
with open(r'C:\Users\Dell\Dropbox\SolBIN\Modelos Python\aRIX engine v2\Inputs\\add_these.txt') as f:
    for line in f.readlines():
        term = line[ : -1] if line[ -1 : ] == '\n' else line[ : ]
        l.append( term )
    f.close()

#adicionando uma lista de termos ao dicinário com os ner_rules
ngrams_to_remove = add_to_ngrams_to_remove(l, ngrams_to_remove, cat = cat_name)

ngrams_to_remove[cat_name]

Adicionando entidades no ngrams_to_replace[extract species]...
Foram adicionados  3  termos (total:  3 )


['B) strain', 'BL strain', 'Bacterial strain']

_______________________________________________________________________________________________________________

## salvando o ngrams_to_remove.json

In [30]:
print('> Classes:')
for key in ngrams_to_remove.keys():
    print(key, '; termos: ', len(ngrams_to_remove[key]))
    
print('Deseja salvar o arquivo ngrams_to_remove.json? (s/n)')
time.sleep(3)
answer = input()
if answer.lower() == 's': 
    with open('ngrams_to_remove.json', 'w') as json_file:
        json.dump(ngrams_to_remove, json_file, indent=2)
        json_file.close()

> Classes:
applications ; termos:  22
extract species ; termos:  3
processes ; termos:  33
publication Year ; termos:  1
raw_materials ; termos:  21
Deseja salvar o arquivo ngrams_to_remove.json? (s/n)


_______________________________________________________________________________________________________________

## manipulando o index_list.json